# Data Analysis With MovieLens 1M Dataset

`GroupLens Research` provides a number of collections of movie ratings data collected from users of MovieLens in the late 1990s and early 2000s. The data provides movie ratings, movie metadata (genres and year), and demographic data about the users (age, zip code, gender identification, and occupation). Such data is often of interest in the development of recommendation systems based on machine learning algorithms.

The MovieLens 1M dataset contains one million ratings collected from six thousand users on four thousand movies. It’s spread across three tables: ratings, user information, and movie information. 

In [1]:
import pandas as pd

In [2]:
a_info = ["user_id", "gender", "age", "occupation", "zip"]

users = pd.read_table("movielens/users.dat", sep="::", header=None, names=a_info,
                      engine="python")

In [3]:
b_info = ["user_id", "movie_id", "rating", "timestamp"]

ratings = pd.read_table("movielens/ratings.dat", sep="::", header=None,
                        names=b_info, engine="python")

In [4]:
c_info = ["movie_id", "title", "genres"]

movies = pd.read_table("movielens/movies.dat", sep="::", header=None,
                       names=c_info, engine="python")

In [5]:
users.head(7)

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810


In [6]:
ratings.head(8)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719


In [7]:
movies.head(4)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama


In [8]:
ratings

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


Analyzing the data spread across three tables is not a simple task; for example, suppose you wanted to compute mean ratings for a particular movie by gender identity and age. As you will see, this is more convenient to do with all of the data merged together into a single table.

In [9]:
data = pd.merge(pd.merge(ratings, users), movies) # I will merge ratings & users then merge result with movie data

In [10]:
data

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...
1000204,5949,2198,5,958846401,M,18,17,47901,Modulations (1998),Documentary
1000205,5675,2703,3,976029116,M,35,14,30030,Broken Vessels (1998),Drama
1000206,5780,2845,1,958153068,M,18,17,92886,White Boys (1999),Drama
1000207,5851,3607,5,957756608,F,18,20,55410,One Little Indian (1973),Comedy|Drama|Western


In [11]:
data.iloc[1]

user_id                                            2
movie_id                                        1193
rating                                             5
timestamp                                  978298413
gender                                             M
age                                               56
occupation                                        16
zip                                            70072
title         One Flew Over the Cuckoo's Nest (1975)
genres                                         Drama
Name: 1, dtype: object

To get mean movie ratings for each film grouped by gender, we can use the `pivot_table` method:

In [12]:
avg_ratings = data.pivot_table("rating", index="title", columns="gender",
                               aggfunc="mean")

In [13]:
avg_ratings.iloc[7:13]

gender,F,M
title,,
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
"13th Warrior, The (1999)",3.112000,3.168000
187 (1997),2.428571,2.791667
2 Days in the Valley (1996),3.488889,3.244813


Group the data by title, and use `size()` to get a Series of group sizes for each title:

In [14]:
ratings_on_title = data.groupby("title").size()

active_titles = ratings_on_title.index[ratings_on_title >= 250]

active_titles

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)',
       ...
       'X-Men (2000)', 'Year of Living Dangerously (1982)',
       'Yellow Submarine (1968)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Sherlock Holmes (1985)',
       'Zero Effect (1998)', 'eXistenZ (1999)'],
      dtype='object', name='title', length=1216)

In [15]:
ratings_on_title.head()

title
$1,000,000 Duck (1971)            37
'Night Mother (1986)              70
'Til There Was You (1997)         52
'burbs, The (1989)               303
...And Justice for All (1979)    199
dtype: int64

The index of titles receiving at least 250 ratings can then be used to select rows from `avg_ratings` using .loc:

In [16]:
avg_ratings = avg_ratings.loc[active_titles]

avg_ratings

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
...,...,...
Young Guns (1988),3.371795,3.425620
Young Guns II (1990),2.934783,2.904025
Young Sherlock Holmes (1985),3.514706,3.363344


To see the top films among female viewers, we can sort by the F column in descending order:

In [17]:
female_ratings = avg_ratings.sort_values("F", ascending=True)

female_ratings

gender,F,M
title,,
Battlefield Earth (2000),1.574468,1.616949
Barb Wire (1996),1.585366,2.100386
Rocky V (1990),1.878788,2.132780
Speed 2: Cruise Control (1997),1.906667,1.863014
"Avengers, The (1998)",1.915254,2.017467
...,...,...
Schindler's List (1993),4.562602,4.491415
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589


## Measuring Rating Disagreement

Suppose you wanted to find the movies that are most divisive between male and female viewers. One way is to add a column to `avg_ratings` containing the difference in means, then sort by that:

In [18]:
avg_ratings["diff"] = avg_ratings["M"] - avg_ratings["F"]

In [19]:
sort_by_diff = avg_ratings.sort_values("diff")

sort_by_diff

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
...,...,...,...
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
"Longest Day, The (1962)",3.411765,4.031447,0.619682
Dumb & Dumber (1994),2.697987,3.336595,0.638608


Suppose instead you wanted the movies that elicited the most disagreement among viewers, independent of gender identification. Disagreement can be measured by the variance or standard deviation of the ratings. To get this, we first compute the rating standard deviation by title and then filter down to the active titles:

In [20]:
rating_std_by_t = data.groupby("title")["rating"].std()

rating_std_by_t = rating_std_by_t.loc[active_titles]

rating_std_by_t

title
'burbs, The (1989)                   1.107760
10 Things I Hate About You (1999)    0.989815
101 Dalmatians (1961)                0.982103
101 Dalmatians (1996)                1.098717
12 Angry Men (1957)                  0.812731
                                       ...   
Young Guns (1988)                    1.017437
Young Guns II (1990)                 1.071959
Young Sherlock Holmes (1985)         0.891176
Zero Effect (1998)                   1.042932
eXistenZ (1999)                      1.178568
Name: rating, Length: 1216, dtype: float64

In [21]:
rating_std_by_t.sort_values(ascending=False)[:10] # 10 most divisively rated movies

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64

You may have noticed that movie genres are given as a pipe-separated `(|)` string, since a single movie can belong to multiple genres. To help us group the ratings data by genre, we can use the `explode` method on DataFrame. 

In [22]:
movies["genres"].iloc[8:16]

8                        Action
9     Action|Adventure|Thriller
10         Comedy|Drama|Romance
11                Comedy|Horror
12         Animation|Children's
13                        Drama
14     Action|Adventure|Romance
15               Drama|Thriller
Name: genres, dtype: object

In [23]:
movies["genres"].head().str.split("|")

0     [Animation, Children's, Comedy]
1    [Adventure, Children's, Fantasy]
2                   [Comedy, Romance]
3                     [Comedy, Drama]
4                            [Comedy]
Name: genres, dtype: object

In [24]:
movies["genre"] = movies.pop("genres").str.split("|")

movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),"[Animation, Children's, Comedy]"
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama]"
4,5,Father of the Bride Part II (1995),[Comedy]


Now, calling `movies.explode("genre")` generates a new DataFrame with one row for each "inner" element in each list of movie genres. For example, if a movie is classified as both a comedy and a romance, then there will be two rows in the result, one with just *"Comedy"* and the other with just *"Romance"*:

In [28]:
movies_exploded = movies.explode("genre")
movies_exploded[32:40]

,movie_id,title,genre
14,15,Cutthroat Island (1995),Romance
15,16,Casino (1995),Drama
15,16,Casino (1995),Thriller
16,17,Sense and Sensibility (1995),Drama
16,17,Sense and Sensibility (1995),Romance
17,18,Four Rooms (1995),Thriller
18,19,Ace Ventura: When Nature Calls (1995),Comedy
19,20,Money Train (1995),Action


Now, we can merge all three tables together and group by genre:

In [31]:
ratings_w_genre = pd.merge(pd.merge(movies_exploded, ratings), users)

ratings_w_genre.iloc[7]

movie_id                   150
title         Apollo 13 (1995)
genre                    Drama
user_id                      1
rating                       5
timestamp            978301777
gender                       F
age                          1
occupation                  10
zip                      48067
Name: 7, dtype: object

In [33]:
genre_ratings = (ratings_w_genre.groupby(["genre", "age"])
                 ["rating"].mean().unstack("age"))

genre_ratings

age,1,18,25,35,45,50,56
genre,,,,,,,
Action,3.506385,3.447097,3.453358,3.538107,3.528543,3.611333,3.610709
Adventure,3.449975,3.408525,3.443163,3.515291,3.528963,3.628163,3.649064
Animation,3.476113,3.624014,3.701228,3.740545,3.734856,3.780020,3.756233
Children's,3.241642,3.294257,3.426873,3.518423,3.527593,3.556555,3.621822
Comedy,3.497491,3.460417,3.490385,3.561984,3.591789,3.646868,3.650949
Crime,3.710170,3.668054,3.680321,3.733736,3.750661,3.810688,3.832549
Documentary,3.730769,3.865865,3.946690,3.953747,3.966521,3.908108,3.961538
Drama,3.794735,3.721930,3.726428,3.782512,3.784356,3.878415,3.933465
Fantasy,3.317647,3.353778,3.452484,3.482301,3.532468,3.581570,3.532700
